In [127]:
import matplotlib.pyplot as plt
import math
import scipy.stats as sps
from numpy.linalg import inv
import csv
import numpy as np
import sys
%matplotlib inline

In [128]:
#Преобразование данных
data = []
with open('forestfires.csv', "rt") as fires:
    reader = csv.reader(fires)
    data = list(reader)
print(data[0])
for elem in data[1:]:
    if elem[2] in ['jun', 'jul', 'aug']:
        elem[2] = 1
    else:
        elem[2] = 0
    elem[3] = 1
data = [[float(x) for x in elem] for elem in data[1:]]
np.random.shuffle(data)

['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain', 'area']


In [129]:
#Для набора и функции для площади пожара сделаем проверку оценки методом линейной регрессии.
def my_estim(dat, func, i_func):     
    sample = dat[:(len(dat) * 7 // 10)]
    new_data = dat[(len(dat) * 7 // 10):]
    square = np.matrix([func(elem[-1]) for elem in sample]).transpose()
    factors = np.matrix([[elem[i] for i in range(len(elem) - 1)] for elem in sample])
    
    coeff = inv(factors.transpose() * factors) * factors.transpose() * square      
    
    new_square = np.matrix([func(elem[-1]) for elem in new_data]).transpose()
    new_factors = np.matrix([[elem[i] 
                        for i in range(len(elem) - 1)] 
                             for elem in new_data])
    estimate_square = new_factors * coeff
    
    diff = np.mean(np.array((estimate_square - new_square).transpose())[0] ** 2)
    inv_diff = np.mean(
        np.array((i_func(estimate_square) - i_func(new_square)).transpose())[0] ** 2)
    return (diff, inv_diff)

In [130]:
print(my_estim(data, (lambda x: x), (lambda x: x)))

(801.33584381522053, 801.33584381522053)


In [131]:
print(my_estim(data, (lambda x: np.log(x + 1)), (lambda x: np.exp(x) - 1)))
print(my_estim(data, (lambda x: np.log(x + 10)), (lambda x: np.exp(x) - 10)))
print(my_estim(data, (lambda x: np.log(x + 100)), (lambda x: np.exp(x) - 100)))
print(my_estim(data, (lambda x: np.log(x + 1000)), 
                    (lambda x: np.exp(x) - 1000)))

(1.9171196025713892, 802.3269768928177)
(0.37987651130117028, 762.25951900774942)
(0.028602313200715691, 723.13756415994055)
(0.00065345852766408765, 748.06952222912753)


In [132]:
#Попробуем найти c, при котором применение f(x) = ln(x+c) даёт наилучшую оценку
result_c = 1
curr_min = 100000
for c in range(1,100):
    if my_estim(data, (lambda x: np.log(x + c)), 
                     (lambda x: np.exp(x) - c))[1] < curr_min:
        curr_min = my_estim(data, (lambda x: np.log(x + c)), 
                                 (lambda x: np.exp(x) - c))[1]
        result_c = c
print(result_c, curr_min)

(99, 723.22258730536976)


Видно, что, даже в лучшем случае функция логарифма даёт очень большое отклонение, порядка 700.

Посмотрим отклонения для разных разбиений:

In [133]:
C = 43
for i in range(15):
    np.random.shuffle(data)
    print(my_estim(data, (lambda x: np.log(x + C)), 
                        (lambda x: np.exp(x) - C)))

(0.19785471792486151, 11979.160793731649)
(0.16466325545675323, 8504.9087736048441)
(0.080319496594456363, 828.76985156018293)
(0.12132967211145573, 7935.1039602110095)
(0.10972567810914056, 3720.152603184838)
(0.086697049317885563, 686.26641996780495)
(0.10933873432690287, 1105.434771989708)
(0.068484857094804716, 422.34504511756717)
(0.12686224058335896, 7928.3786219495769)
(0.089630227210422728, 705.92881799641373)
(0.15367251724823153, 8382.5759592946779)
(0.13708699041904629, 4274.0749280252057)
(0.096314485754443332, 913.75558730316823)
(0.096844037799529742, 809.0047517172037)
(0.087594058500510677, 697.35662399448142)


Видно, что наше изначальное разбиение можно даже назвать вполне удачным, т.к. при других отклонение может вырастать в разы.

Можно сделать вывод, что в данном наборе наш метод оценки неэффективен.